In [1]:
pip install tdqm

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install librosa

In [3]:
pip install tensorflow

  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.3.0 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.6.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [4]:
# Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
%matplotlib inline
import seaborn as sns

# Specific Libraries
import os
import librosa
import librosa.display

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm, trange
from tqdm.auto import tqdm

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

import IPython.display as ipd

In [ ]:
pip install tdqm

In [ ]:
df = pd.read_csv("UrbanSound8K.csv")
df.head()

In [ ]:
dat1, sampling_rate1 = librosa.load('C:\\Users\\sonaw\\Documents\\study purpose\\mini project 2\\urbanexdata\\fold5\\65381-3-0-7.wav')


In [ ]:
dat2, sampling_rate2 = librosa.load('C:\\Users\\sonaw\\Documents\\study purpose\\mini project 2\\urbanexdata\\fold5\\6508-9-0-1.wav')


In [ ]:
plt.figure(figsize=(20, 20))
D = librosa.amplitude_to_db(np.abs(librosa.stft(dat1)), ref=np.max)
plt.subplot(4, 2, 1)
librosa.display.specshow(D, y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')

In [ ]:
dat2

In [ ]:
dat1

In [ ]:
plt.figure(figsize=(20, 20))
D = librosa.amplitude_to_db(np.abs(librosa.stft(dat2)), ref=np.max)
plt.subplot(4, 2, 1)
librosa.display.specshow(D, y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')

In [ ]:
pip install resampy


In [ ]:
def features_extract(file):
    sample,sample_rate = librosa.load(file_name,res_type='kaiser_fast')
    feature = librosa.feature.mfcc(y=sample,sr=sample_rate,n_mfcc=50)
    scaled_feature = np.mean(feature.T,axis=0)
    return scaled_feature

extracted = []
path = "C:\\Users\\sonaw\\Downloads\\UrbanEc"

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['slice_file_name'])) 
    final_class_labels = row['class']   
    data= features_extract(file_name)    
    extracted.append([data,final_class_labels])

In [ ]:
ext_df = pd.DataFrame(extracted,columns=['feature','class'])
ext_df

In [ ]:
ext_df.to_csv('extracteddf.csv', index=False)

In [ ]:
x = np.array(ext_df['feature'].tolist())
y = np.array(ext_df['class'].tolist())

In [ ]:
le = LabelEncoder()

y = to_categorical(le.fit_transform(y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

print("Number of training samples = ", x_train.shape[0])
print("Number of testing samples = ",x_test.shape[0])

In [ ]:
num_labels = 10  # Replace with the actual number of labels in your dataset

model1 = Sequential()
#num_labels = y.shape[1]
model1 = Sequential()

model1.add(Dense(128, input_shape=(50,)))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))

model1.add(Dense(256))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(256))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(128))

model1.add(Dense(num_labels))
model1.add(Activation('softmax'))

model1.summary()

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="C:\\Users\\sonaw\\model1.hdf5",
                               verbose=1, save_best_only=True)
model1.fit(
          x_train, 
          y_train, 
          batch_size=50, 
          epochs=200,
          validation_data=(x_test, y_test),callbacks=[checkpointer]
         )

In [ ]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T,axis=0)
    return np.array([scaled])

In [ ]:
def predict(file_name):
    pred_fea = extract_feature(file_name) 
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    return pred_class[0]

In [ ]:
df

In [ ]:
test_accuracy=model1.evaluate(x_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
predict("C:\\Users\\sonaw\\Downloads\\UrbanEc\\fold5\\6508-9-0-1.wav")
ipd.Audio("C:\\Users\\sonaw\\Downloads\\UrbanEc\\fold5\\6508-9-0-1.wav")

In [ ]:
df['class'].unique()

In [ ]:
file1="C:\\Users\\sonaw\\Downloads\\UrbanEc\\fold5\\6508-9-0-1.wav"
predict(file2)
ipd.Audio(file2)

In [ ]:
pip install sounddevice

In [ ]:
import sounddevice as sd

# Define the duration of the recording in seconds
duration = 5  # Adjust as needed

# Sample rate (number of samples per second)
sample_rate = 22050  # Adjust as needed

# Record sound
print("Recording started. Speak into the microphone...")
recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
print(recording)
# Wait for the recording to complete
sd.wait()

# Access the recorded sound data
sound_data = recording.flatten()   


# Example: Print the length of the sound data
print("Sound data length:", len(sound_data))

# Call the predict function
prediction = predict(sound_data)
print("Prediction:", prediction)
extract_feature(sound_data)

In [ ]:
predict(sound_data)
ipd.Audio(sound_data)

In [ ]:
sampling_rate1

In [ ]:
pip install typing-extensions==3.10.0.2


In [ ]:
pip install --upgrade tensorflow-intel


In [ ]:
pip install gradio

In [ ]:
import gradio as gr

# Define your Gradio interface and functions here

if __name__ == "__main__":
    gr.Interface(...).launch()


In [ ]:
inp = gr.inputs.Audio(type='filepath',label = 'Audio File')
demo = gr.Interface(fn =predict,inputs = inp , outputs ="text")
demo.launch(share = True,debug = True)
demo